# Image classification

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
import sys

from tensorflow.contrib.layers import fully_connected, convolution2d, flatten, batch_norm, max_pool2d, dropout
from tensorflow.contrib.layers import l2_regularizer, l1_regularizer
from tensorflow.python.ops.nn import relu, softmax
from tensorflow.python.framework.ops import reset_default_graph

from batcher import Batcher

In [ ]:
def onehot(t, n_classes):
    out = np.zeros((t.shape[0], n_classes))
    for row, col in enumerate(t):
        out[row, col] = 1
    return out

In [ ]:
batcher = Batcher('text', rep='textdata')

In [ ]:
examples, labels = batcher.get_batch(58,10)
examples = (1+examples)/2
plt.figure(figsize=(15,8))
for i, im in enumerate(examples):
    plt.subplot(2,5,i+1)
    plt.imshow(im)
    plt.title(labels[i])
plt.show()

## Model

In [ ]:
def fit_batch(sess, batch):
    x_batch, y_batch = batch
    fetches = [train_step]
    feed = {x_pl: x_batch, y_pl: onehot(y_batch, n_classes), is_training: True}
    sess.run(fetches, feed)
    
    
def eval_batch(sess, batch):
    x_batch, y_batch = batch
    fetches = [cross_entropy, accuracy]
    feed = {x_pl: x_batch, y_pl: onehot(y_batch, n_classes), is_training: False}
    batch_loss, batch_acc = sess.run(fetches, feed)
    
    return batch_loss, batch_acc

def predict_batch(sess, batch):
    x_batch, y_batch = batch
    fetches = [y]
    feed = {x_pl: x_batch, is_training: False}
    res = sess.run(fetches, feed)
    return res[0]

In [ ]:
def fit(sess, batcher, n_samples_train=15000, eval_train=True, track_valid = True, n_samples_valid=5000, \
        n_epochs=10, batch_size=100, early_stopping=True):
    """Fits the model to the training data.
    If track_valid=True, the model will be evaluated on the validation data too"""
    
    n_batches_train = n_samples_train // batch_size
    
    if eval_train:
        train_acc, train_loss = [], []
        
    if track_valid:
        valid_acc, valid_loss = [], []
        n_batches_valid = n_samples_valid // batch_size
        
    if early_stopping:
        patience = 3
        cnt = 0
    
    for epoch in range(n_epochs):
        
        batcher.reset_iterator()
        
        print('Epoch', epoch+1)
        print('Training:', end=" ")
        
        for i in range(n_batches_train):
            batch = batcher.next_batch(batch_size)
            fit_batch(sess, batch)
            if (i+1)%10==0:
                print(i+1, end=" ")
        
        print()
        
        if eval_train:
            batcher.reset_iterator()
            cur_loss = 0
            cur_acc = 0
            print('Evaluating model on training set...', end=' ')
            for i in range(n_batches_train):
                batch = batcher.next_batch(batch_size)
                batch_loss, batch_acc = eval_batch(sess, batch)
                cur_loss += batch_loss
                cur_acc += batch_acc
                if (i+1)%10==0:
                    print(i+1, end=" ")

            train_loss.append(cur_loss/n_batches_train)
            train_acc.append(cur_acc/n_batches_train)
        
        if track_valid:
            print()
            cur_loss = 0
            cur_acc = 0
            print('Evaluating model on validation set...', end=' ')
            for i in range(n_batches_valid):
                batch = batcher.next_batch(batch_size)
                batch_loss, batch_acc = eval_batch(sess, batch)
                cur_loss += batch_loss
                cur_acc += batch_acc
                if (i+1)%10==0:
                    print(i+1, end=" ")
                    
            if early_stopping and len(valid_loss) > 1 and valid_loss[-1] > valid_loss[-2]:
                cnt += 1
                    
            print()

            valid_loss.append(cur_loss/n_batches_valid)
            valid_acc.append(cur_acc/n_batches_valid)
        
        if eval_train:
            print('train loss =', train_loss[-1], 'train acc =', train_acc[-1])
        if track_valid:
            print('valid loss =', valid_loss[-1], 'valid acc =', valid_acc[-1])
            
        if track_valid and early_stopping and cnt == patience:
                print('Early stopping')
                break
    
    return train_loss, train_acc, valid_loss, valid_acc

In [ ]:
def plot_train_valid(train_values, valid_values):
    xaxis = np.arange(len(train_values))
    plt.plot(xaxis, train_values, c='b', label='Training')
    plt.plot(xaxis, valid_values, c='r', label='Validation')
    plt.legend(loc='best')
    
def plot_loss_acc(res):
    train_loss, train_acc, valid_loss, valid_acc = res
    
    plt.figure(figsize=(12,4))
    
    plt.subplot(121)
    plot_train_valid(train_loss, valid_loss)
    plt.title('loss')
    
    plt.subplot(122)
    plot_train_valid(train_acc, valid_acc)
    plt.title('accuracy')
    
    plt.show()

In [ ]:
def save(sess, path):
    saver = tf.train.Saver()
    saver.save(sess, path)
    
def restore(sess, path, filename):
    saver = tf.train.import_meta_graph(path+filename)
    saver.restore(sess, tf.train.latest_checkpoint(path))

In [ ]:
def resmodule(input_tensor, n_filters, reg_const=0.1):
    
    conv1 = convolution2d(input_tensor, n_filters, kernel_size=[3,3], activation_fn=relu, \
                          weights_regularizer=l2_regularizer(reg_const), \
                          normalizer_fn=None, padding='SAME')
    conv2 = convolution2d(conv1, n_filters, kernel_size=[3,3], activation_fn=None, \
                          weights_regularizer=l2_regularizer(reg_const),\
                          normalizer_fn=None, padding='SAME')
    
    return relu(input_tensor+conv2)

def transition_resmodule(input_tensor, n_filters_in, n_filters_out, reg_const=0.1):
    """The original ResNet paper considers both zero-padding and projection shortcuts when the dimension increases.
    We do only zero-padding, as the performance is said to be almost the same with lower complexity."""
    
    conv1 = convolution2d(input_tensor, n_filters_out, kernel_size=[3,3], stride = 2, activation_fn=relu, \
                          weights_regularizer=l2_regularizer(reg_const), \
                          normalizer_fn=None, padding='SAME')
    conv2 = convolution2d(conv1, n_filters_out, kernel_size=[3,3], activation_fn=None, \
                          weights_regularizer=l2_regularizer(reg_const), \
                          normalizer_fn=None, padding='SAME')
    
    # since we change the spatial dimension, we must take the input with stride 2 (original paper too)
    strided = input_tensor[:,::2,::2,:]
    
    n_zeros = n_filters_out - n_filters_in
    paddings = [[0,0], [0,0], [0,0], [0, n_zeros]]
    padded = tf.pad(strided, paddings, "CONSTANT")  
    
    return relu(padded + conv2)


def module(input_tensor, n_filters, n_layers=2, kernel=[3,3], stride=1, reg_const=0.1):
    
    conv = convolution2d(input_tensor, n_filters, kernel_size=kernel, stride=stride, activation_fn=relu, \
                         weights_regularizer=l2_regularizer(reg_const), padding='SAME')
    
    for i in range(n_layers-1):
        conv = convolution2d(conv, n_filters, kernel_size=kernel, stride=stride, activation_fn=relu, \
                             weights_regularizer=l2_regularizer(reg_const), padding='SAME')

    pool = max_pool2d(conv, kernel_size=[2,2])
    
    return pool

In [ ]:
h, w, d = 128, 128, 3
n_classes = 2
kernel = [3,3]
stride = [1,1]
reg = 1.0

reset_default_graph()

x_pl = tf.placeholder(tf.float32, [None, h, w, d])
is_training = tf.placeholder(tf.bool)

"""
mod1 = module(x_pl, 32, n_layers=3, kernel=kernel, stride=stride, reg_const=reg)
mod2 = module(mod1, 64, n_layers=3, kernel=kernel, stride=stride, reg_const=reg)
mod3 = module(mod2, 128, n_layers=2, kernel=kernel, stride=stride, reg_const=reg)

l_flatten = flatten(mod3)
l_dropout = dropout(l_flatten, keep_prob=0.5, is_training=is_training)

l1 = fully_connected(l_dropout, 256, activation_fn=relu, weights_regularizer=l2_regularizer(reg))
l1_dropout = dropout(l1, keep_prob=0.5, is_training=is_training)

y = fully_connected(l1_dropout, n_classes, activation_fn=softmax)
"""

conv1 = convolution2d(x_pl, 32, kernel_size=[5,5], stride=2, activation_fn=relu, \
                         weights_regularizer=l2_regularizer(reg), padding='SAME')
pool1 = max_pool2d(conv1, kernel_size=[2,2])

mod = resmodule(pool1, 32, reg)
mod = resmodule(mod, 32, reg)

mod = transition_resmodule(mod, 32, 64, reg)
mod = resmodule(mod, 64, reg)

mod = transition_resmodule(mod, 64, 128, reg)

pool2 = max_pool2d(mod, kernel_size=[2,2])

y = fully_connected(flatten(pool2), n_classes, activation_fn=softmax)

In [ ]:
y_pl = tf.placeholder(tf.float32, [None, n_classes])

cross_entropy = -tf.reduce_sum(y_pl * tf.log(y+1e-8))
cross_entropy = tf.reduce_mean(cross_entropy)

loss = cross_entropy

optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_pl, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

In [ ]:
res = fit(sess, batcher, n_epochs=10, n_samples_train=8000, n_samples_valid=2000, early_stopping=False)

In [ ]:
save(sess, 'xsave/3mod_20ep')

In [ ]:
plot_loss_acc(res)

In [ ]:
imgs, true_labels = batcher.next_batch(48)
preds = predict_batch(sess, (imgs, true_labels))
imgs = (1+imgs)/2

In [ ]:
plt.figure(figsize=(15,40))

for i,pred in enumerate(preds):
    plt.subplot(12,4,i+1)
    plt.imshow(imgs[i])
    s = 'Vrai : %d, Probabilité X: %.02f' % (true_labels[i], pred[1])
    plt.title(s)
plt.show()